# 📊 **Strategy Report Builder – Sales Transcript to Insightful Report**


---


This notebook uses Google Gemini (`genai`) to analyze sales conversations and extract a structured strategy report including client needs, proposed solutions, objections, next steps, and strategic insights.


---



## 🧩 **1. Install Required Libraries**
Ensure the required Google GenAI Python SDK is installed.

In [1]:
!pip install -q google-generativeai -q

## 🔐 **2. Configure API Key**
Set your Google API key (keep it secret!).

In [2]:
# Replace the empty string with your actual API key. This key is used for authentication with the Google Generative AI..
import os
os.environ['GEMINI_API_KEY'] = "your-actual-api-key-here"


## 📥**3. Import Libraries**
We’ll import all nec`essary modules from the Google GenAI SDK.

In [3]:
from google import genai
from google.genai import types
import os

## 🧠 **4. Define the Strategy Report Agent**
This function analyzes the transcript and generates a structured report.

In [12]:
# 🧠 Sales Report Agent
# This function analyzes a sales transcript and generates a structured strategy report using Google Gemini

def sales_report(transcript):
    # Step 1: Define the system-level instruction for the model
    # This sets the persona of the model as a business analyst focusing on extracting insights from sales conversations
    strategy_system_prompt = """
    You are a strategic business analyst specializing in interpreting sales conversations.
    Your role is to extract actionable insights, structure them into a report, and ensure clarity,
    completeness, and business relevance. Focus on summarizing client needs, solutions offered,
    objections, next steps, and strategic takeaways.
    """

    # Step 2: Define the dynamic user prompt with placeholders for the transcript
    # This gives explicit instructions to the model on what kind of output is expected
    user_prompt = """
    You are a Strategy Report Builder.

    Your job is to analyze a sales conversation transcript between a salesperson and a client,
    and generate a clear, actionable, and structured strategy report based on the discussion.

    Your report should include the following sections:
    1. **Client Needs & Pain Points**
    2. **Proposed Solutions**
    3. **Objections or Concerns Raised**
    4. **Next Steps & Follow-ups**
    5. **Strategic Insights**

    Output only the report in a professional, structured format.
    Do not repeat the transcript or add explanations.

    Transcript:
    {{transcript}}
    """

    # Step 3: Inject the actual transcript into the placeholder
    user_prompt = user_prompt.replace("{{transcript}}", transcript)

    # Step 4: Configure Gemini with the system instruction as a context
    config = types.GenerateContentConfig(system_instruction=strategy_system_prompt)

    # Step 5: Create a Gemini client with the API key and configuration
    sales_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"), config=config)

    # Step 6: Generate the content using the specified Gemini model and composed user prompt
    response = sales_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=user_prompt
    )

    # Step 7: Return the model's response in a dictionary for further use or formatting
    return {"strategy_report": response.text}


## 🛠️**5. Define Tool Schema**
This allows Gemini to call the sales report function as a tool.

In [5]:
# 🛠️ Define the Sales Report Tool Schema
# This declares a tool that can be called by the model using structured function-calling syntax

sales_report_tool = {
    "name": "sales_report",  # The name of the function/tool the model will call
    "description": "Create a sales report for a given transcript",  # Human-readable purpose
    "parameters": {
        "type": "object",  # Parameters passed as a JSON object
        "properties": {
            "transcript": {
                "type": "string",
                "description": "Sales transcript"  # What kind of input is expected
            }
        },
        "required": ["transcript"]  # Enforces that 'transcript' must be provided
    }
}

# 🔧 Wrap the tool in a Gemini Tool object
# This enables the model to call the tool when it's relevant to the user’s prompt
tools = types.Tool(
    function_declarations=[sales_report_tool]
)

# ⚙️ Attach the tool configuration to the generation config
# This allows Gemini to recognize and use the function in the context of chat generation
config = types.GenerateContentConfig(
    tools=[tools]
)


## 📤 **6. Provide a Sample Sales Transcript**
Gemini will parse this and decide whether to call the appropriate tool.

In [8]:
# 📝 User Input: Sales Conversation Prompt
# This is the initial message coming from the user, simulating a request to analyze a transcript.
# The structure uses Gemini's `types.Content` and `types.Part` format to send model-ready input.

contents = [
    types.Content(
        role="user",  # Specifies that this input comes from the user
        parts=[
            types.Part(
                text='''create a sales report for the transcript:

                Salesperson: Hi Sarah, thanks for taking the time today. Just to confirm, you're looking to improve your customer onboarding process, right?
                Client: Yes, that's right. Our current flow is clunky, and users are dropping off before completing setup.
                Salesperson: Got it. Based on what you shared earlier, I think our OnboardPro tool could help streamline that experience. It’s reduced drop-offs by over 40% for similar clients.
                Client: That sounds promising. What would the integration look like?
                Salesperson: It’s a low-code plugin, so your dev team can integrate it in under a week. We also offer support during setup.
                Client: Okay, and what's the pricing structure?
                Salesperson: It’s usage-based. For your estimated volume, it would be around $2,000/month.
                Client: That might be a stretch for our current budget. Are there tiered options?
                Salesperson: Absolutely. We can start with the Lite package at $1,200/month and upgrade later as your usage grows.
                Client: That’s more feasible. Can you send over a detailed proposal?
                Salesperson: Of course. I’ll send it today. Would you be open to a follow-up call next Wednesday to walk through it?
                Client: Yes, Wednesday works. Thanks!
                '''
            )
        ]
    )
]


## 🤖**7. Execute the Agent and Generate the Strategy Report**

In [10]:
# 🚀 Initialize the Gemini Client
# This client instance is used to send requests to the Gemini API.
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# 🎯 Step 1: Initial Response Generation
# Send the user's request and tool definitions to the model.
# The model may respond with either direct output or a function call.
response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=config,
    contents=contents
)

# 🧠 Step 2: Check if the model made a function call
# We access the first candidate's first part to inspect whether a tool was invoked.
tool_calls = response.candidates[0].content.parts[0].function_call

# ✅ Step 3: If a function call was made, handle it
if tool_calls:
    if tool_calls.name == "sales_report":  # Only handle the "sales_report" tool here

        # 🛠️ Execute the corresponding function with the provided transcript
        tool_response = sales_report(tool_calls.args['transcript'])

        # 🔄 Prepare the tool response part to be passed back to Gemini
        function_response_part = types.Part.from_function_response(
            name=tool_calls.name,
            response={"result": tool_response},  # Wrap response in a result object
        )

        # 🧱 Step 4: Build the conversation chain
        # Add the tool call made by the model
        contents.append(types.Content(role="model", parts=[types.Part(function_call=tool_calls)]))

        # Add the tool's response to the conversation
        contents.append(types.Content(role="tool", parts=[function_response_part]))

        # 🗣️ Step 5: Regenerate response with updated conversation history
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=config,
            contents=contents
        )

        # 🖨️ Display final output from the model
        print(response.text)

else:
    # If no tool was called, print the direct model response
    print(response.text)


OK. I've created a sales report for the transcript you provided. Here's the strategy report:

## Strategy Report

**1. Client Needs & Pain Points**

*   Inefficient and clunky customer onboarding process.
*   High user drop-off rates during the setup phase.
*   Budget constraints.

**2. Proposed Solutions**

*   Implementation of OnboardPro tool to streamline customer onboarding.
*   Low-code plugin integration for quick deployment.
*   Tiered pricing options (Lite package) to align with budget.

**3. Objections or Concerns Raised**

*   Initial pricing ($2,000/month) perceived as too high.

**4. Next Steps & Follow-ups**

*   Send a detailed proposal to the client outlining the OnboardPro solution.
*   Schedule a follow-up call for next Wednesday to review the proposal.

**5. Strategic Insights**

*   Client is budget-conscious, making the Lite package a strategic entry point.
*   Highlight the ease of integration (low-code) to address potential technical concerns.
*   Quantify the va